# Convex Hull

![convex hull](https://upload.wikimedia.org/wikipedia/commons/thumb/d/de/ConvexHull.svg/440px-ConvexHull.svg.png)

## Convex Hull Problem Description

We are creating a convex polygon: https://en.wikipedia.org/wiki/Convex_polygon

The Convex Hull problem is a fundamental problem in computational geometry that deals with finding the smallest convex polygon that contains all the points in a given set. In simple terms, imagine you have a set of points on a plane, and you want to create a rubber band that stretches around these points such that it encloses all of them. When the rubber band is released, it will contract and wrap around the outermost points, forming the convex hull of the set.

Mathematically, a set of points S in a Euclidean space is said to be convex if for every pair of points P and Q in S, the line segment connecting P and Q lies entirely within S. The convex hull is the smallest convex set containing all the points in the given set.

In computational geometry, the Convex Hull problem has several applications, such as collision detection in video games, path planning in robotics, and shape analysis in computer vision.

There are various algorithms to solve the Convex Hull problem, some of which are:

* Graham's Scan: This algorithm sorts the points by their polar angles and processes them in a stack to build the convex hull. Its time complexity is O(n log n), as the most time-consuming step is sorting the points.

* Gift Wrapping Algorithm (Jarvis March): This algorithm starts with the leftmost point and iteratively selects the next point by wrapping around the set in a counter-clockwise direction. Its time complexity is O(nh), where n is the number of input points, and h is the number of points in the convex hull.


* QuickHull: This algorithm is a recursive divide-and-conquer approach, similar to the QuickSort algorithm. It selects extreme points and divides the input set into subsets based on their position relative to the line connecting the extreme points. The time complexity of QuickHull is O(n log n) in the average case, but it can degrade to O(n^2) in the worst case.

* Chan's Algorithm: This algorithm is a combination of Graham's Scan and Jarvis March. It computes the convex hull in O(n log h) time, making it an optimal output-sensitive algorithm.



Each of these algorithms has its strengths and weaknesses, and the choice of which one to use depends on factors such as the input size, expected output size, and available computational resources.


## Graham's Scan

Graham's Scan is an efficient algorithm for finding the convex hull of a set of points in a 2D plane. It has a time complexity of O(n log n), where n is the number of input points. The algorithm follows these steps:

* Find the lowest y-coordinate point (P0):
Identify the point with the lowest y-coordinate among the input points. If there are multiple points with the same y-coordinate, choose the one with the lowest x-coordinate. This point (P0) is guaranteed to be a part of the convex hull.

* Sort the points based on their polar angles with respect to P0:
Calculate the polar angles of the remaining points with respect to P0. Sort the points in increasing order of these polar angles. In case of a tie (when two points have the same polar angle), order them based on their distance from P0, with the closest point coming first.

* Initialize the convex hull stack:
Create a stack data structure to store the points in the convex hull. Push the first three sorted points (P0, P1, and P2) onto the stack.

* Process the remaining sorted points:
Iterate through the remaining sorted points, starting from the fourth point (P3). For each point, perform the following steps:

1. Check the orientation of the triplet formed by the top two points in the stack and the current point. If the orientation is counter-clockwise (a left turn), it means the current point is a valid convex hull point. Push the current point onto the stack and proceed to the next point in the sorted list.

2. If the orientation is clockwise (a right turn) or collinear, it means the second-to-top point in the stack is not part of the convex hull, as it is "inside" the turn. Pop the top point off the stack and recheck the orientation of the new triplet formed by the top two points in the stack and the current point. Repeat this process until a counter-clockwise turn is encountered or the stack has only two points left. Then, push the current point onto the stack and continue to the next point in the sorted list.

Termination:
Once all the points have been processed, the points remaining in the stack form the vertices of the convex hull in counter-clockwise order.

The most time-consuming step in Graham's Scan is sorting the points based on their polar angles, which takes O(n log n) time. The actual construction of the convex hull using the stack has a linear complexity of O(n). Therefore, the overall time complexity of the algorithm is O(n log n).

In [1]:
import math

def lowest_y_coordinate(points):
    return min(points, key=lambda point: (point[1], point[0]))

def polar_angle(p0, p1):
    return math.atan2(p1[1] - p0[1], p1[0] - p0[0])

def distance(p0, p1):
    return (p1[0] - p0[0])**2 + (p1[1] - p0[1])**2

def orientation(p0, p1, p2):
    return (p1[1] - p0[1]) * (p2[0] - p0[0]) - (p1[0] - p0[0]) * (p2[1] - p0[1])

def graham_scan(points):
    '''This implementation takes a list of 2D points as input, 
    where each point is represented as a tuple of (x, y) coordinates.
      It returns a list of points representing the vertices of the convex hull in counter-clockwise order.

Please note that this code does not handle duplicate points or points with the same polar angle. 
To handle these edge cases, you can modify the sorting step accordingly.'''
    if len(points) < 3:
        return points

    # Step 1: Find the lowest y-coordinate point
    lowest_point = lowest_y_coordinate(points)
    
    # Step 2: Sort the points based on their polar angles with respect to the lowest_point
    sorted_points = sorted(points, key=lambda point: (polar_angle(lowest_point, point), distance(lowest_point, point)))
    
    # Step 3: Initialize the convex hull stack
    convex_hull = [sorted_points[0], sorted_points[1], sorted_points[2]]

    # Step 4: Process the remaining sorted points
    for i in range(3, len(sorted_points)):
        while len(convex_hull) >= 2 and orientation(convex_hull[-2], convex_hull[-1], sorted_points[i]) <= 0:
            convex_hull.pop()
        convex_hull.append(sorted_points[i])

    return convex_hull

# Example usage:
points = [(1, 1), (2, 5), (3, 3), (5, 3), (3, 2), (2, 2)]
convex_hull = graham_scan(points)
print(convex_hull)

[(1, 1), (2, 5)]


In [2]:
points = [(1, 1), (1, 5), (5, 1), (5, 5), (3, 3), (2, 2)]
convex_hull = graham_scan(points)
print(convex_hull)

[(1, 1), (5, 1), (2, 2), (1, 5)]


## Improved Graham's Scan to handle collinear points



In [3]:
import math

def lowest_y_coordinate(points):
    return min(points, key=lambda point: (point[1], point[0]))

def polar_angle(p0, p1):
    return math.atan2(p1[1] - p0[1], p1[0] - p0[0])

def distance(p0, p1):
    return (p1[0] - p0[0])**2 + (p1[1] - p0[1])**2

def orientation(p0, p1, p2):
    return (p1[1] - p0[1]) * (p2[0] - p0[0]) - (p1[0] - p0[0]) * (p2[1] - p0[1])

def graham_scan(points):
    if len(points) < 3:
        return points

    # Step 1: Find the lowest y-coordinate point
    lowest_point = lowest_y_coordinate(points)
    
    # Step 2: Sort the points based on their polar angles and distance with respect to the lowest_point
    sorted_points = sorted(points, key=lambda point: (polar_angle(lowest_point, point), distance(lowest_point, point)))

    # Remove duplicate points
    sorted_points = [sorted_points[i] for i in range(len(sorted_points)) if i == 0 or sorted_points[i] != sorted_points[i - 1]]
    
    # Step 3: Initialize the convex hull stack
    convex_hull = [sorted_points[0], sorted_points[1], sorted_points[2]]

    # Step 4: Process the remaining sorted points
    for i in range(3, len(sorted_points)):
        while len(convex_hull) >= 2 and orientation(convex_hull[-2], convex_hull[-1], sorted_points[i]) <= 0:
            convex_hull.pop()
        convex_hull.append(sorted_points[i])

    return convex_hull

# Example usage:
points = [(1, 1), (2, 5), (3, 3), (5, 3), (3, 2), (2, 2), (3, 3), (5, 3)]
convex_hull = graham_scan(points)
print(convex_hull)

[(1, 1), (2, 5)]


## Visualization of Graham's Scan


In [4]:
import plotly.graph_objs as go

# def lowest_y_coordinate(points):
#     # ... (same as before)

# def polar_angle(p0, p1):
#     # ... (same as before)

# def distance(p0, p1):
#     # ... (same as before)

# def orientation(p0, p1, p2):
#     # ... (same as before)

def graham_scan_trace(points):
    if len(points) < 3:
        return points, []

    # Step 1: Find the lowest y-coordinate point
    lowest_point = lowest_y_coordinate(points)
    
    # Step 2: Sort the points based on their polar angles and distance with respect to the lowest_point
    sorted_points = sorted(points, key=lambda point: (polar_angle(lowest_point, point), distance(lowest_point, point)))

    # Remove duplicate points
    sorted_points = [sorted_points[i] for i in range(len(sorted_points)) if i == 0 or sorted_points[i] != sorted_points[i - 1]]
    
    # Step 3: Initialize the convex hull stack
    convex_hull = [sorted_points[0], sorted_points[1], sorted_points[2]]
    convex_hull_trace = []

    # Step 4: Process the remaining sorted points
    for i in range(3, len(sorted_points)):
        while len(convex_hull) >= 2 and orientation(convex_hull[-2], convex_hull[-1], sorted_points[i]) <= 0:
            convex_hull.pop()
        convex_hull.append(sorted_points[i])
        convex_hull_trace.append(go.Scatter(x=[point[0] for point in convex_hull] + [convex_hull[0][0]],
                                            y=[point[1] for point in convex_hull] + [convex_hull[0][1]],
                                            mode="lines",
                                            line=dict(color="gray", dash="dash"),
                                            name=f"Step {i+1}"))

    return convex_hull, convex_hull_trace

def visualize(points, convex_hull, convex_hull_trace):
    input_points = go.Scatter(
        x=[point[0] for point in points],
        y=[point[1] for point in points],
        mode="markers",
        marker=dict(size=8, color="blue"),
        name="Input Points",
    )

    hull_points = go.Scatter(
        x=[point[0] for point in convex_hull] + [convex_hull[0][0]],
        y=[point[1] for point in convex_hull] + [convex_hull[0][1]],
        mode="lines+markers",
        marker=dict(size=8, color="red"),
        line=dict(color="red"),
        name="Convex Hull",
    )

    traces = [input_points, hull_points] + convex_hull_trace
    layout = go.Layout(title="Graham's Scan Algorithm Visualization", showlegend=True)

    fig = go.Figure(data=traces, layout=layout)
    fig.show()

points = [(1, 1), (2, 5), (2.5, 3), (5, 3), (3, 2), (2, 2), (3, 3.5), (5, 3)]
convex_hull, convex_hull_trace = graham_scan_trace(points)
visualize(points, convex_hull, convex_hull_trace)

## Geeks for Geeks version of Graham's

src: https://www.geeksforgeeks.org/convex-hull-using-graham-scan/

In [14]:
# A Python3 program to find convex hull of a set of points. Refer
# https://www.geeksforgeeks.org/orientation-3-ordered-points/
# for explanation of orientation()
 
from functools import cmp_to_key
 
# A class used to store the x and y coordinates of points
class Point:
    def __init__(self, x = None, y = None):
        self.x = x
        self.y = y

    def __str__(self):
        return f"{self.x},{self.y}"

    def __repr__(self):
        return f"{self.x},{self.y}"
 
# A global point needed for sorting points with reference
# to the first point
p0 = Point(0, 0)
 
# A utility function to find next to top in a stack
def nextToTop(S):
    return S[-2]
 
# A utility function to return square of distance
# between p1 and p2
def distSq(p1, p2):
    return ((p1.x - p2.x) * (p1.x - p2.x) +
            (p1.y - p2.y) * (p1.y - p2.y))
 
# To find orientation of ordered triplet (p, q, r).
# The function returns following values
# 0 --> p, q and r are collinear
# 1 --> Clockwise
# 2 --> Counterclockwise
def orientation(p, q, r):
    val = ((q.y - p.y) * (r.x - q.x) -
           (q.x - p.x) * (r.y - q.y))
    if val == 0:
        return 0  # collinear
    elif val > 0:
        return 1  # clock wise
    else:
        return 2  # counterclock wise
 
# A function used by cmp_to_key function to sort an array of
# points with respect to the first point
def compare(p1, p2):
   
    # Find orientation
    o = orientation(p0, p1, p2)
    if o == 0:
        if distSq(p0, p2) >= distSq(p0, p1):
            return -1
        else:
            return 1
    else:
        if o == 2:
            return -1
        else:
            return 1
 
# Prints convex hull of a set of n points.
def convexHull(points, n):
   
    # Find the bottommost point
    ymin = points[0].y
    min = 0
    for i in range(1, n):
        y = points[i].y
 
        # Pick the bottom-most or choose the left
        # most point in case of tie
        if ((y < ymin) or
            (ymin == y and points[i].x < points[min].x)):
            ymin = points[i].y
            min = i
 
    # Place the bottom-most point at first position
    points[0], points[min] = points[min], points[0]
 
    # Sort n-1 points with respect to the first point.
    # A point p1 comes before p2 in sorted output if p2
    # has larger polar angle (in counterclockwise
    # direction) than p1
    p0 = points[0]
    points = sorted(points, key=cmp_to_key(compare))
 
    # If two or more points make same angle with p0,
    # Remove all but the one that is farthest from p0
    # Remember that, in above sorting, our criteria was
    # to keep the farthest point at the end when more than
    # one points have same angle.
    m = 1  # Initialize size of modified array
    for i in range(1, n):
       
        # Keep removing i while angle of i and i+1 is same
        # with respect to p0
        while ((i < n - 1) and
        (orientation(p0, points[i], points[i + 1]) == 0)):
            i += 1
 
        points[m] = points[i]
        m += 1  # Update size of modified array
 
    # If modified array of points has less than 3 points,
    # convex hull is not possible
    if m < 3:
        return
 
    # Create an empty stack and push first three points
    # to it.
    S = []
    S.append(points[0])
    S.append(points[1])
    S.append(points[2])
 
    # Process remaining n-3 points
    for i in range(3, m):
       
        # Keep removing top while the angle formed by
        # points next-to-top, top, and points[i] makes
        # a non-left turn
        while ((len(S) > 1) and
        (orientation(nextToTop(S), S[-1], points[i]) != 2)):
            S.pop()
        S.append(points[i])
 
    # Now stack has the output points,
    # print contents of stack
    # while S:
    #     p = S[-1]
    #     print("(" + str(p.x) + ", " + str(p.y) + ")")
    #     S.pop()
    return S
 
# Driver Code
input_points = [(0, 3), (1, 1), (2, 2), (4, 4),
                (0, 0), (1, 2), (3, 1), (3, 3)]
points = []
for point in input_points:
    points.append(Point(point[0], point[1]))
n = len(points)
convex_points = convexHull(points, n)
convex_points  #so without print we needed to overload __repr__

[0,0, 3,1, 4,4, 0,3]

In [13]:
print(convex_points[0])

0,0


In [15]:
def visualize_simple(points, convex_hull, convex_hull_trace=()):
    input_points = go.Scatter(
        x=[point[0] for point in points],
        y=[point[1] for point in points],
        mode="markers",
        marker=dict(size=8, color="blue"),
        name="Input Points",
    )

    hull_points = go.Scatter(
        x=[point[0] for point in convex_hull] + [convex_hull[0][0]],
        y=[point[1] for point in convex_hull] + [convex_hull[0][1]],
        mode="lines+markers",
        marker=dict(size=8, color="red"),
        line=dict(color="red"),
        name="Convex Hull",
    )

    traces = [input_points, hull_points] + list(convex_hull_trace)
    layout = go.Layout(title="Graham's Scan Algorithm Visualization", showlegend=True)

    fig = go.Figure(data=traces, layout=layout)
    fig.show()

In [18]:
convex_points_list = [(t.x,t.y) for t in convex_points]
convex_points_list

[(0, 0), (3, 1), (4, 4), (0, 3)]

In [22]:
points[0], type(points[0])

(0,0, __main__.Point)

In [23]:
visualize_simple(input_points, convex_points_list) # we do not pass traces for now

## Jarvis March - aka - Gift Wrapping Algorithm


Jarvis March, also known as the Gift Wrapping algorithm, is a method for finding the convex hull of a set of points in a 2D plane. It is called the Gift Wrapping algorithm because its process is similar to wrapping a gift with a piece of string by iteratively stretching it around the outermost points of the gift. The algorithm has a time complexity of O(nh), where n is the number of input points and h is the number of points in the convex hull.

Here's a detailed description of how the Jarvis March algorithm works:

* Find the leftmost point: Start by identifying the point with the lowest x-coordinate, which is guaranteed to be part of the convex hull. If there are multiple points with the same x-coordinate, choose the one with the lowest y-coordinate. This point is referred to as the "anchor" point.

* Initialize the current point: Set the current point as the anchor point.

* Find the next hull point: For each point in the input set, excluding the current point, determine the point that forms the smallest clockwise angle (also known as the smallest polar angle) with the current point and the positive x-axis (or the line segment connecting the current point and the previous hull point, after the first iteration). In other words, find the point that would create a right turn if we were to move from the previous hull point to the current point and then to the candidate point. The point with the smallest clockwise angle becomes the next hull point.

* Check for termination: If the next hull point is the anchor point, the algorithm terminates, and the convex hull is complete. If not, set the current point as the next hull point and repeat step 3.

* Convex hull: The sequence of points visited in the clockwise direction forms the convex hull.

The Jarvis March algorithm is relatively simple to understand and implement. However, its time complexity depends on the number of points in the convex hull (h). In the worst case, when all points are part of the convex hull, the algorithm has a time complexity of O(n^2). This makes it less efficient than other algorithms like Graham's Scan or Chan's Algorithm for larger input sets or when the convex hull has many points. However, Jarvis March can be more efficient for cases where the convex hull has a small number of points, regardless of the total number of input points.



![Gift](https://upload.wikimedia.org/wikipedia/commons/9/9c/Animation_depicting_the_gift_wrapping_algorithm.gif)

In [24]:
def leftmost_point(points):
    return min(points, key=lambda point: (point[0], point[1]))

def orientation(p0, p1, p2):
    return (p1[1] - p0[1]) * (p2[0] - p0[0]) - (p1[0] - p0[0]) * (p2[1] - p0[1])

def jarvis_march(points):
    if len(points) < 3:
        return points

    # Step 1: Find the leftmost point
    anchor = leftmost_point(points)

    # Initialize the convex hull
    convex_hull = [anchor]

    # Step 2: Initialize the current point
    current_point = anchor

    while True:
        # Step 3: Find the next hull point
        next_point = None
        for candidate_point in points:
            if candidate_point == current_point:
                continue
            if next_point is None or orientation(current_point, next_point, candidate_point) > 0:
                next_point = candidate_point

        # Step 4: Check for termination
        if next_point == anchor:
            break

        # Update the current point and add the next hull point to the convex hull
        current_point = next_point
        convex_hull.append(next_point)

    return convex_hull

# Example usage:
points = [(1, 1), (2, 5), (3, 3), (5, 3), (3, 2), (2, 2)]
convex_hull = jarvis_march(points)
print(convex_hull)

[(1, 1), (5, 3), (2, 5)]


In [25]:
points = [(1, 1), (1, 5), (3, 3), (5, 5), (2, 3), (5,1)]
convex_hull = jarvis_march(points)
print(convex_hull)

[(1, 1), (5, 1), (5, 5), (1, 5)]


## Visualizing the Jarvis March Algorithm


In [26]:
def jarvis_march_trace(points):
    if len(points) < 3:
        return points, []

    # ... (same as before)
    anchor = leftmost_point(points)
    convex_hull = [anchor]
    current_point = anchor
    convex_hull_trace = []

    while True:
        next_point = None
        for candidate_point in points:
            if candidate_point == current_point:
                continue
            if next_point is None or orientation(current_point, next_point, candidate_point) > 0:
                next_point = candidate_point

        if next_point == anchor:
            break

        current_point = next_point
        convex_hull.append(next_point)
        convex_hull_trace.append(go.Scatter(x=[point[0] for point in convex_hull] + [convex_hull[0][0]],
                                            y=[point[1] for point in convex_hull] + [convex_hull[0][1]],
                                            mode="lines",
                                            line=dict(color="gray", dash="dash"),
                                            name=f"Step {len(convex_hull)}"))

    return convex_hull, convex_hull_trace

def visualize(points, convex_hull, convex_hull_trace):
    input_points = go.Scatter(
        x=[point[0] for point in points],
        y=[point[1] for point in points],
        mode="markers",
        marker=dict(size=8, color="blue"),
        name="Input Points",
    )

    hull_points = go.Scatter(
        x=[point[0] for point in convex_hull] + [convex_hull[0][0]],
        y=[point[1] for point in convex_hull] + [convex_hull[0][1]],
        mode="lines+markers",
        marker=dict(size=8, color="red"),
        line=dict(color="red"),
        name="Convex Hull",
    )

    traces = [input_points, hull_points] + convex_hull_trace
    layout = go.Layout(title="Jarvis March Algorithm Visualization", showlegend=True)

    fig = go.Figure(data=traces, layout=layout)
    fig.show()

points = [(1, 1), (2, 5), (3, 3), (5, 3), (3, 2), (2, 2)]
convex_hull, convex_hull_trace = jarvis_march_trace(points)
visualize(points, convex_hull, convex_hull_trace)

In [27]:
points = [(1, 1), (2, 3), (4, 5), (5, 7), (6, 1), (2, 2),(3,3),(3,0.5)]
convex_hull, convex_hull_trace = jarvis_march_trace(points)
visualize(points, convex_hull, convex_hull_trace)

In [28]:
import random
points = [(random.randint(1,100)/10, random.randint(1,100)/10) for _ in range(30)]
convex_hull, convex_hull_trace = jarvis_march_trace(points)
visualize(points, convex_hull, convex_hull_trace)

## Chan's Algorithm and other approaches

There are other algorithms for solving Convex Hull such as Chan's algorithm

They generally use divide and conquer approach but and guarantee complexity O(n log h) - which on average will beat other algorithms

![Chan](https://upload.wikimedia.org/wikipedia/commons/thumb/4/48/ChanAlgDemo.gif/300px-ChanAlgDemo.gif)

Src: https://en.wikipedia.org/wiki/Chan%27s_algorithm